In [ ]:
import nltk
import pandas as pd
import re

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
import nltk
import pandas as pd
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sujaikarunakaran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sujaikarunakaran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
df=pd.read_csv(r'/Users/sujaikarunakaran/Desktop/Assignments/Spring 2020/Ranga/Final Project/Final/RateMds/Phrases_Positive.csv')


In [7]:
df_positive=pd.read_csv(r'/Users/sujaikarunakaran/Desktop/Assignments/Spring 2020/Ranga/Final Project/Final/RateMds/Phrases_Positive.csv')



In [ ]:
df = df[df.Sentiment != "Neutral"]

In [ ]:
df_negative = df[df.Sentiment == "Negative"]
df_positive = df[df.Sentiment == "Positive"]

In [ ]:
df_positive = df[df.Sentiment == "Positive"]

In [ ]:
#df_negative = df[df_negative]
#df_positive = df[df_positive]

In [ ]:
df_positive.info()

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)

In [8]:
doctor_names=pd.read_csv(r'/Users/sujaikarunakaran/Desktop/Assignments/Spring 2020/Ranga/Final Project/doctor_names_ratemds.csv')



In [9]:
first_names = doctor_names["first_names"].to_list()
middle_names = doctor_names["midle_names"].to_list()
last_names = doctor_names["last_names"].to_list()

In [10]:
first_names = [x for x in first_names if str(x) != 'nan']
middle_names = [x for x in middle_names if str(x) != 'nan']
last_names = [x for x in last_names if str(x) != 'nan']

In [11]:
names = first_names + middle_names + last_names

In [12]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [13]:
stop_words.extend(names)

In [14]:
stop_words.extend(['dr', 'Dr'])

In [15]:
stop_words.extend(['doctor'])

In [16]:
stop_words.extend(['dr', 'shashoua', 'great', 'drachler', 'excellent', 'brilliant', 'terrible', 'awful'])

In [17]:
stop_words.extend(['roth', 'ramsey', 'bodner', 'ob_gyn', 'absolutely', 'recommend', 'Roth'])
stop_words.extend(['Lipowich', 'lipowich', 'gyne-oncologist'])

In [18]:
len(stop_words)

2020

In [19]:
df_positive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21137 entries, 0 to 21136
Data columns (total 3 columns):
Doc_names         21137 non-null object
Review_phrases    21137 non-null object
Sentiment         21137 non-null object
dtypes: object(3)
memory usage: 495.5+ KB


In [31]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [32]:
df['Review_phrases'] = list(sent_to_words(df['Review_phrases']))

In [33]:
bigram = gensim.models.Phrases(df['Review_phrases'], min_count=5, threshold=100)

In [34]:
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [35]:
print(bigram_mod[df_negative['Review_phrases'])

SyntaxError: invalid syntax (<ipython-input-35-17a528472f11>, line 1)

In [23]:
df['Review_phrases'] = df['Review_phrases'].map(lambda x: x.lstrip("\''"))



In [24]:
df['Review_phrases'] = df['Review_phrases'].map(lambda x: x.lstrip("%"))

In [25]:
df['Review_phrases'] = df['Review_phrases'].map(lambda x: x.strip("-"))

In [26]:
df['Review_phrases'] = df['Review_phrases'].map(lambda x: x.strip(')('))

In [27]:
df['Review_phrases'] = df['Review_phrases'].map(lambda x: x.strip('"'))

In [28]:
df['Review_phrases'] = df['Review_phrases'].str.replace('\d+', '')

In [29]:
df['Review_phrases'] = df['Review_phrases'].map(lambda x: x.lower())

In [30]:
df['Review_phrases']

0                                    he is simply the best
1                           drachler is very knowledgeable
2                   his bedside manners are simply perfect
3         drachler is an excellent example of how a phy...
4                                        'it was my first 
5         michael drachler is god-sent directly from he...
6                                               competent 
7                                  i trust him completely 
8                                         drachler greatly
9         he not only loves to bring new life into the ...
10                                          to be healthy 
11                         drachler is truly one of a kind
12                           she was pregnant within  days
13                                 drachler is very caring
14              will not end the visit until you are ready
15        drachler is one of the best doctors i have ev...
16         as you know most doctors see you for - minute

In [36]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [ ]:
def remove_stop_words(text):
    for doc in text:
        review_words = ""
        if doc not in stop_words:
            review_words = doc
        return review_words

In [37]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [38]:
df['Review_phrases'] = remove_stopwords(df['Review_phrases'])

In [ ]:
txt = remove_stopwords(["He pushes his appoints back sometimes as much as 2 hours"])

In [ ]:
txt

In [ ]:
df['Review_phrases']

In [39]:
df['Review_phrases'] = make_bigrams(df['Review_phrases'])

In [ ]:
df_positive['review_removed'] = lemmatization(df_positive['review_removed'])

In [ ]:
df['Review_phrases']

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [ ]:
df_positive["Review_phrases"] = remove_stopwords(df_positive["Review_phrases"])

In [ ]:
df_positive["Review_phrases"]

In [40]:
df['texts'] = [', '.join(map(str, l)) for l in df['Review_phrases']]

In [ ]:
df['texts']

In [ ]:
df_negative = df[df.Sentiment == "Negative"]
df_positive = df[df.Sentiment == "Positive"]

In [45]:
vectorizer = vectorizer.fit(df['texts'])
tfidf = vectorizer.transform(df['texts'])
vocab = vectorizer.get_feature_names()
print(len(vocab))

1229


In [ ]:
vocab

In [ ]:
pip install corextopic

In [46]:
from corextopic import corextopic as ct

In [64]:
anchors = []
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)

In [67]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=20)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: feel, comfortable, feel comfortable, sure, makes, questions, made, make, made feel, make sure, makes feel, answer, easy talk, makes sure, things, answer questions, made sure, feel like, always, talk
Topic #2: time, pregnant, get, appointment, wait, first time, weeks, felt, took, see, back, minutes, went, much, told, takes, months, call, day, times
Topic #3: ever, best, would highly, highly, best ever, would, one best, highly recommended, recommended, anyone, bed_side, one, best ob_gyn, highly anyone, best doctors, ob_gyn, bed_side manner, far best, obgyn, ever seen
Topic #4: first, baby, pregnancy, delivered, child, years, new, many, first pregnancy, first child, healthy, many years, first visit, deliver, high_risk, delivered first, son, first baby, ob, delivering
Topic #5: staff, office staff, office, friendly, staff friendly, humor, sense, nice, sense humor, staff wonderful, staff always, staff also, also, staff nice, staff amazing, good experience, always friendly, really 

In [49]:
bow=pd.read_csv(r'/Users/sujaikarunakaran/Desktop/Assignments/Spring 2020/Ranga/Final Project/Final/RateMds/Buckets_ v1.csv')

In [ ]:
bow

In [50]:
med_expertise = bow["Medical Expertise"].to_list()

In [51]:
bedside_manners = bow["Bedside Manners"].to_list()
office_staff = bow["Staff"].to_list()
costs = bow["Costs/Expenses"].to_list()
clinic_envt = bow["Office Environment"].to_list()
ease_schedule = bow["Ease of Appointment"].to_list()
waiting_time = bow["Waiting Time"].to_list()

In [ ]:
office_management = office_staff + costs + clinic_envt

In [58]:
#cleanedList = [x for x in countries if str(x) != 'nan']
bedside_manners = [x for x in bedside_manners if str(x) != 'nan']

In [53]:
office_staff = [x for x in office_staff if str(x) != 'nan']
costs = [x for x in costs if str(x) != 'nan']
clinic_envt = [x for x in clinic_envt if str(x) != 'nan']
ease_schedule = [x for x in ease_schedule if str(x) != 'nan']
waiting_time = [x for x in waiting_time if str(x) != 'nan']

In [54]:
costs.extend(["pay", "amount", "bill", "billing", "cash", "charge", "charging", "insurance", "medicaid", "medic", "aid"])

In [59]:
anchors = med_expertise, bedside_manners, office_staff, costs, clinic_envt, ease_schedule, waiting_time
#anchors = med_expertise, bedside_manners, office_management, ease_schedule, waiting_time

In [60]:
len(anchors[1])

165

In [57]:
bedside_manners.extend(["bedside_manner", "rude", "bedside_manners", "bedside", "manners", "manner"])

In [61]:
anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

In [62]:
model = ct.Corex(n_hidden=7, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=50 # Tell the model how much it should rely on the anchors
)

In [63]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=30)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: pregnant, knowledgeable, able, practice, child, delivered, surgery, delivery, health, job, issues, work, medical, birth, life, section, labor, deliver, knowledge, exam, check, options, delivering, test, ultrasound, pregnancies, field, decision, treatment, complications
Topic #2: time, feel, care, comfortable, amazing, patient, questions, bedside_manner, talk, willing, pleasant, help, answer, gentle, attentive, unprofessional, concerns, manner, confident, listen, answers, explain, bed_side, respectful, explains, question, answered, explained, informative, special
Topic #3: staff, nurse, nurses, information, service, nursing, team, front, dealing, assistant, staff friendly, staff wonderful, staff always, staff also, staff nice, staff amazing, nursing staff, also, nice, staff awesome, staff helpful, wonderful staff, friendly staff, staff kind, staff well, also friendly, staff best, wonderful, always friendly, well
Topic #4: insurance, pay, rate, paid, amount, bill, charge, billi

In [ ]:
df_positive = df_positive.drop(['topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7'], axis = 1)



In [ ]:
df_positive

In [ ]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(7)]
).astype(float)
topic_df.index = df_negative.index
df_negative = pd.concat([df_negative, topic_df], axis=1)

In [ ]:
df_negative.sample(10, random_state=1)

In [ ]:
df_positive[20:30]

In [ ]:
df_new = df_negative.rename(columns={'topic_1': 'med_expertise_neg', 'topic_2':'bedside_manners_neg', 'topic_3':'office_staff_neg', 'topic_4':'costs_neg', 'topic_5':'clinic_envt_neg', 'topic_6':'ease_schedule_neg', 'topic_7':'waiting_time_neg'})
                              
                              
                              
                              
                              
                              
                    

In [ ]:
df_new

In [ ]:
df_new.to_csv(r'/Users/sujaikarunakaran/Desktop/Assignments/Spring 2020/Ranga/Final Project/Final/Healthgrades/Negative Phrases/Negative_AS8_NW30.csv', header=True, index=False)


In [ ]:
###############################Labelled LDA############################

In [ ]:
df1=pd.read_csv(r'/Users/sujaikarunakaran/Downloads/Phrase_with_Sentiment_V1.csv')



In [ ]:
df1_positive = df_positive

In [ ]:
df1_negative = df_negative

In [ ]:
df1_negative['Review_phrases'] = df1_negative['Review_phrases'].map(lambda x: x.strip('"'))

In [ ]:
df1_positive['Review_phrases'] = df1_positive['Review_phrases'].map(lambda x: x.strip("-"))

In [ ]:
df1_positive['Review_phrases'] = df1_positive['Review_phrases'].map(lambda x: x.strip('][)('))

In [ ]:
df1_positive['Review_phrases'] = df1_positive['Review_phrases'].map(lambda x: x.strip("\''"))

In [ ]:
df1_positive['Review_phrases'] = df1_positive['Review_phrases'].str.replace('\d+', '')

In [ ]:
df1_positive['Review_phrases'] = df1_positive['Review_phrases'].map(lambda x: x.lower())

In [ ]:
df1_positive['Review_phrases']

In [ ]:
bigram = gensim.models.Phrases(df1_positive['Review_phrases'], min_count=5, threshold=100) 
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [ ]:
df1_positive['Review_phrases'] = list(sent_to_words(df1_positive['Review_phrases']))

In [ ]:
df1_positive['review_removed'] = remove_stopwords(df1_positive['Review_phrases'])

In [ ]:
df1_positive['review_removed'] = make_bigrams(df1_positive['review_removed'])

In [ ]:
df1_positive['review_removed']

In [ ]:
id2word = corpora.Dictionary(df1_positive['Review_phrases'])

In [ ]:
id2word

In [ ]:
texts = df1_positive['Review_phrases']

In [ ]:
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
print(corpus[:1])

In [ ]:
id2word[0]

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df1_positive['Review_phrases'], dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [ ]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '/Users/sujaikarunakaran/Downloads/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
pprint(ldamallet.show_topics(formatted=False))

In [ ]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=df1_positive['review_removed'], dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=df1_positive['review_removed'], start=2, limit=40, step=6)



In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))